# 偏旁规范化以及去重

偏旁规范化，指的是让辅助码更符合 `zrm2000` 的映射规则，以此降低一个字同时对应多组辅助码的问题，从而减少重码。

- **部首** 与 *偏旁* 的区别: [高的偏旁部首为什么是高(高是偏旁部首吗) - 儿童教育](http://www.xychild.com/zhongxiaoxue/shuokepingke/5750.html)

- 如果一个字由部首(可以理解为"主要"的偏旁)和其它部分(也是偏旁), "自然码+辅助码"的第三码取做部首, 第四码取为最后一个偏旁.

- 如果一个字本身就是部首, 即独体字, 不能进一步拆分出部件, 只能由笔画构成. "自然码+辅助码"的第三码是第一笔, 第四码是最后一笔.

`zrm2000` 的键位可参考以下链接：

1. https://www.liuchuo.net/archives/2847

2. https://zhuanlan.zhihu.com/p/122866844

3. 键位图 https://blog.csdn.net/pmo992/article/details/104963648


如果一个文件(比如 `zrm-pinyin.dict.yaml`) 存在一个字同一发音多个辅码的情况，比如说 `星` 字，有 `xy;ou` 和 `xy;ru` 两个码，则可以通过

1. 用 `reShape` 函数对 `zrm-pinyin.dict.yaml` 操作 `xy;ru -> xy;ou`，输出至文件 `zrm-pinyin.temp.dict.yaml`。

2. 用 `rmRepeat` 函数对 `zrm-pinyin.temp.dict.yaml` 操作干掉重复的 entry，输出至文件 `zrm-pinyin.wanted.dict.yaml`。

3. 备份原来的 `zrm-pinyin.dict.yaml`，再手动重命名 `zrm-pinyin.wanted.dict.yaml -> zrm-pinyin.dict.yaml` ，rime就会拿这个新文件当字典来用

## 1. 拆字准备

需要下载的外部 python package: 

1. [miaoluda/hanzi_chaizi forked from howl-anderson/hanzi_chaizi](https://github.com/miaoluda/hanzi_chaizi/tree/patch-1)。请注意要下载的是 fork 版 **不要下载原版** ，否则以下必报错

导入 `sys`, `HanziChaizi` 并且熟悉后者的拆字逻辑。

### 1.1 有明显部首的字

HanziChaizi 可以拆出大多数 *偏旁*, 但结果的第0个元素不一定是 **部首** :

- '伍' 拆字之后的第0个元素是 '人', 是部首.
- '变' 拆字之后的第0个元素是 '亦', 是偏旁, *不是部首*.

In [3]:
import sys

# 外部包 miaoluda/hanzi_chaizi 默认放到 scripts 目录之下
# 如果报错, 就把下面这行换成你下载的 miaoluda/hanzi_chaizi 的本地位置:
sys.path.append("./hanzi_chaizi")
from hanzi_chaizi import HanziChaizi

hc = HanziChaizi()
# 测试以下拆字结果，比如 亻 旁，输出的就是个正常的 “人”。注意这种输出的匹配，
# 不要不加测试就盲目的拿 “亻” 去下面的代码 run，那样会毫无效果
result = hc.query('伍')

print(result)
print(hc.query('变'))

['人', '五']
['亦', '又']


### 1.2 独体字
一般是部首汉字，如：“金木水火土辶皿马皮日月目衣耳”等。独体字全部看成部首，不能进一步拆分出部件，只能由笔画构成。
对于这类字, 不要用 HanziChaizi 的结果!

In [2]:
hc.query('由')

['囗', '十']

### 1.3 ”拆不动“的字

大部分为独体字或者 HanziChaizi 未收录的字, 输出 `None`

In [3]:
print(hc.query('一'))

None


对一些极少使用的字有误伤，后果不严重。

In [4]:
print(hc.query('鯈'))
print(hc.query('擜'))

['攸', '魚']
None


In [11]:
print(hc.query('膀'))
print(hc.query('臂'))
print(hc.query('瞥'))
print(hc.query('撻'))

['肉', '旁']
['肉', '辟']
['敝', '目']
['手', '達']


## 2. 构造用于分析文件的函数

主要思路：for循环 + if条件判断。

`reShape()` 函数用于规范化第三码(部首码，也就是分号后面第一辅助码), 目前暂未考虑分号后第二辅助码，以后可能再出个 `reShape2()` 用来改进。

`rmRepeat()` 函数用于去除“完全重复的编码”。

In [1]:
def reShape(src, temp):
    # 重新map辅助码中的 部首 
    # src: "input dict" ; dst: "new file name for the output dict"
    with open(src, "r", encoding="utf-8") as f1:
        with open(temp, "w", encoding="utf-8") as f2:
            for line in f1:
                    
                # 如果缺失换行符 \n，则添加 \n.
                if '\n' not in line: 
                    line2 = line + '\n'  
                
                # line[0] 不是汉字，或者“不能拆”
                if line[0].isascii() or hc.query(line[0]) is None: 
                    # 该条件已包含以'\n'开头的空行
                    line2 = line
                
                # line[0] 是“可以拆”的汉字，开始拆偏旁
                # line[5] 就是每行分号后的第一辅码（下称第三码）
                # 第三码为 ;r，则可能为 双立人，单立人，日，其他
                elif line[5]=='r':
                    # 判为'行'字旁(左)，第三码 r -> x
                    if hc.query(line[0])[0]=='彳':
                        line2 = line.replace(';r', ';x')
                    # 判为'人'字旁(左)，第三码原样输出 ;r。
                    # e.g. 为了防止"傝"字（ = 人+日+etc）被改成 ;o
                    elif hc.query(line[0])[0]=='人':
                        line2 = line
                    # 判为'日'字旁，第三码 r -> o
                    elif '日' in hc.query(line[0]):
                        line2 = line.replace(';r', ';o')
                    # 剩下的 ;r 原样输出
                    else: 
                        line2 = line
                # 判为'月'字旁 第三码 y -> o
                elif line[5]=='y' and '肉' in hc.query(line[0]):
                    line2 = line.replace(';y', ';o')
                # 判为'目'字旁 第三码 m -> o
                elif line[5]=='m' and '目' in hc.query(line[0]):
                    line2 = line.replace(';m', ';o')
                # 判为扶'手'旁(左) 第三码 t -> f
                elif line[5]=='t' and hc.query(line[0])[0]=='手':
                    line2 = line.replace(';t', ';f')
                # 判为'亠'旁(上) 第三码 d -> w
                elif line[5]=='d' and hc.query(line[0])[0]=='亠':
                    line2 = line.replace(';d', ';w')
                # 剩下的所有不关心的，原样输出
                else: 
                    line2 = line
                    
                f2.write(line2)

def reShape2(temp0, temp):
    # 计划整理第四码，not finished
    pass
                
def rmRepeat(temp, dst):
    # 去重。
    # src: "input dict" ; dst: "new file name for the output dict"
    with open(temp, "r", encoding="utf-8") as f1:
        with open(dst, "w", encoding="utf-8") as f2:
            last_line = '\n'
            for line in f1:
                if line == last_line:
                    f2.write('')
                else:
                    f2.write(line)
                last_line = line


## 3. 分析文件

In [5]:
# temp.yaml 里面有“更符合规范”的码，但也有重复条目，因为 reShape() 没有删除任何一行！
reShape('../zrm_pinyin.standard_unique.dict.yaml', 'temp.yaml')   

# 去重, 并且(可选)覆盖原来的文件. 这样得到有“更符合规范”的码，且无重复条目
rmRepeat('temp.yaml', '../zrm_pinyin.standard_unique.dict.yaml')

经过上述过程, 第三码(部首/首个偏旁)得到了部分纠正, 而第四码(末个偏旁)未被调整. 

所以肯定仍然存在不准确的码. 欢迎提issue, 以便完善筛选部首的函数, 以及为第四码的错码找规律. 

## 4. 可能有错的编码

- 独体字, 如前所述, HanziChaizi 拆不准
    - 需要手动修改, 比如 '文', '方', '立' 等字, 应该按笔画拆分, 而不是提取 '亠' 作为部首.
- 生僻字, HanziChaizi 拆不了 (输出 None)
- 有部首的汉字, 但是拆错了部首. 比如"仅", 可能错把偏旁"又"当成了部首(正确的部首应该是"人").